<h2>Generating a higher resolution shelf for a portion of the American Eastern Seaboard using NOAA's 3 arc-second Coastal Relief Model:</h2>

Downloaded v2 of the NCRM from here: https://www.ngdc.noaa.gov/mgg/coastal/grddas02/grddas02.htm
as xyz triads.

Vitals:
S.E. Atlantic coverage: N to 48°; S to 24°; E from -65°; and W to -107°   EPSG::4269   (GCS on NAD83)

NOAA's Coastal Relief Model (NCRM) resolution is 3 arc-seconds  (0.0008333°)

<h3>Here's how to do it, step by step:</h3>

<h3>Create new location and mapset in GRASS:</h3>

In [ ]:
location = N_America_GCS_EPSG4269
mapset = user

<h3>Extract the bathymetry (or the topography) from the coastal relief model topo-bathy point cloud [using sqlite]:</h3>
This step is optional. 

Because the downloaded xyz file is 3.65 GB working with it as a whole could prove problematic. So, you can use the UNIX command line and the command 'split' to break the file into 14 10 million line, non-overlapping parts (NOTE: I ended up not using the split files approach...)

**From the UNIX Command Line:**

In [ ]:
wc -l se_atl_crm_v1.xyz
s 133302258 se_atl_crm_v1.xyz     133,302,258 lines

Break file into about 14 files, each of 10 million lines with prefix: NRCM_v2_
split  -l 10000000 se_atl_crm_v1.xyz NCRM_v2_

<h3>Working with a bigger data file using SQLite:</h3>

In [ ]:
### Appealing the SQLite, load the se_atl_crm_v1.xyz to a table, then select elevation values <= 0 


### To extract bathymetry only, and save to external file:

bash# sqlite3 norm_v2.sqlite
> create table tbl(x real, y real, z real);
> .separator " "
> .import se_atl_crm_v1.xyz tbl
> .mode csv
> .output NCRM_v2_bathy.csv
> select * from tbl where z <= 0.0;
> .quit

### To extract a complementary topographic point cloud:

bash# sqlite3 norm_v2.sqlite
> create table tbl(x real, y real, z real);
> .separator " "
> .import se_atl_com_v1.xyz tbl
> .mode csv
> .output NCRM_v2_topo.csv
> select * from tbl where z > 0
> .quit

<h3>Generate topo-bathy Digital Surface Model from the coastal relief model point cloud:</h3>

**in GRASS, get the geographic extents of the data (you'll use this to create a suitable computational region for the resulting topo-bathy surface model):**

In [ ]:
### NOTE: use the -s flag with r.in.xyz to restrict command to finding extents only.

r.in.xyz -s --overwrite input=/Users/paulp/shelves_data/se_atl/se_atl_crm_v1.xyz output=NCRM_NAmer_v2_xyz_3sec@user type=CELL separator=space

###    x_min: -83       y_max: -68.0
###    y_min: 30.0      y_max: 41.0
###    z_min: -4369.2   z_max: 0.0

**Set the computational region to encompass the v2 point cloud at 3 arc seconds- horizontal and 1-meter vertical):**

In [ ]:
g.region --overwrite n=41.0 s=30.0 e=-68.0 w=-83.0 res=0.000833333 tbres=1.0 save=NCRM_ENAmerica_V2_3sec

**Import the NCRM point cloud to create a raster [point] surface:**

In [ ]:
r.in.xyz --overwrite input=/Users/paulp/shelves_data/se_atl/se_atl_crm_v1.xyz output=NCRM_NAmer_v2_xyz_3sec@user type=CELL separator=space percent=75

**Create a mask around the raster points in NCRM_NAmer_v2_xyz_3sec  to restrict computations during the upcoming resampling/interpolation procs:**

In [ ]:
r.to.vect --overwrite input=NCRM_NAmer_v2_xyz_3sec@user output=NCRM_NAmer_v2_xyz type=point
v.hull --overwrite input=NCRM_NAmer_v2_xyz@user output=NCRM_v2_xyz_chull
v.to.rast --overwrite input=NCRM_v2_xyz_chull@user output=NCRM_NAmer_v2_msk_3sec@user use=cat

**Resample and interpolate (using rst) w/o changing resolutions to craft the final topo-bathy DEM:**

In [ ]:
r.mask NCRM_NAmer_v2_msk_3sec
r.resamp.rst --overwrite input=NCRM_NAmer_v2_xyz_3sec@user ew_res=0.0008333 ns_res=0.0008333 elevation=NCRM_NAmer_v2_topobathy_3sec

r.mask -r

** Extract the shelf-only bathymetry from the topobathy DSM (optional, but it might help):**

Note: this works for extracting the topography, too. Just change the names, as appropriate.

In [ ]:
r.mapcalc --o
NCRM_NAmer_v2_shelfbathy_3sec = if(  (NCRM_NAmer_v2_topobathy_3sec < 0 ) && ( NCRM_NAmer_v2_topobathy_3sec  >= -1211 ), 1, null() )
end

<h3>Computing the geomorphon forms and ternary pattern layers and extract the shelf surface model:</h3>

**Fit geomorphon types:**

In [ ]:
r.geomorphon --overwrite elevation=NCRM_NAmer_v2_topobathy_3sec forms=NCRM_NAmer_v2_forms_3sec ternary=NCRM_NAmer_v2_ternary_3sec search=5 skip=0 flat=0.69 dist=0 step=0 start=0

**Extract a pre-shelf (where the geomorphon type and depth criteria, but not continuity, are applied to the topobathy data) layer:**

In [ ]:
r.mapcalc --o
NCRM_NAmer_v2_preshelf_3sec=if( (NCRM_NAmer_v2_forms_3sec == 1) && (NCRM_NAmer_v2_topobathy_3sec <0.0 && NCRM_NAmer_v2_topobathy_3sec >=-1211), 1, null() )
end

**Create a pre shelf bathy layer from the just-created preshelf form layer:**

In [ ]:
r.mapcalc --o
NCRM_NAmer_v2_preshelfbathy_3sec = if( NCRM_NAmer_v2_preshelf_3sec == 1, NCRM_NAmer_v2_topobathy_3sec, null() )
end

**Create a topography only layer from the NCRM topobathy data:**

In [ ]:
r.mapcalc --o
NCRM_NAmer_v2_topo_3sec = if( NCRM_NAmer_v2_topobathy_3sec >= 0.0, NCRM_NAmer_v2_topobathy_3sec, null() )
end

**Use r.lake to apply the final (continuity) criterion to create the final shelf layer:**

In [ ]:
r.lake --overwrite elevation=NCRM_NAmer_v2_preshelf_3sec@user water_level=2.0 lake=NCRM_NAmer_v2_shelf_3sec@user seed=NCRM_NAmer_v2_topo_3sec@user

**Congratulations. You now have a continental, or insular, shelf!**